In [9]:
from pathlib import Path
from zipfile import ZipFile
from tqdm import tqdm
import pandas as pd
import boto3
from dotenv import load_dotenv
import os
import io

In [10]:
# Chargement API KEYS
load_dotenv()

API_KEY_S3 = os.environ["API_KEY_S3"]
API_SECRET_KEY_S3 = os.environ["API_SECRET_KEY_S3"]

bucket_name = "renergies99-bucket"
s3_prefix = "public" 

In [11]:
# Session Boto3
session = boto3.Session(
    aws_access_key_id=API_KEY_S3,
    aws_secret_access_key=API_SECRET_KEY_S3,
    region_name="eu-west-3",
)

s3 = session.resource("s3")
bucket = s3.Bucket(bucket_name)

# Dossier S3 avec données
national_prefix = f"{s3_prefix}/prod/unzipped/national/"

# Lister les objets S3 sous ce préfixe
xlsx_keys = sorted(
    obj.key
    for obj in bucket.objects.filter(Prefix=national_prefix)
    if obj.key.endswith(".xlsx")
)

print(f"{len(xlsx_keys)} fichiers trouvés sur S3")

# 2. Charger et concaténer
dfs = []
for key in xlsx_keys:
    s3_obj = s3.Object(bucket_name, key).get()
    buffer = io.BytesIO(s3_obj["Body"].read())

    df = pd.read_excel(buffer)  # ici seulement xlsx → pas besoin de gérer xls
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

print(df_all.shape)
df_all.head()

14 fichiers trouvés sur S3
(486734, 40)


,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
0,France,Données définitives,2012-01-01,00:00:00,58315.0,58200.0,58200.0,492.0,25.0,3816.0,...,ND,ND,ND,ND,ND,ND,NaN,NaN,NaN,NaN
1,France,Données définitives,2012-01-01,00:15:00,NaN,57700.0,57550.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,France,Données définitives,2012-01-01,00:30:00,58315.0,57200.0,56900.0,492.0,25.0,3816.0,...,ND,ND,ND,ND,ND,ND,NaN,NaN,NaN,NaN
3,France,Données définitives,2012-01-01,00:45:00,NaN,56200.0,56000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,France,Données définitives,2012-01-01,01:00:00,56231.0,55200.0,55100.0,492.0,25.0,3834.0,...,ND,ND,ND,ND,ND,ND,NaN,NaN,NaN,NaN


In [12]:
df_all = df_all[~df_all["Heures"].astype(str).str.endswith(("15:00", "45:00"))]

In [ ]:
# Colonnes à convertir en float (exclusions)
cols_float = [
    c for c in df_all.columns
    if c not in ["Périmètre","Nature","Date", "Heures"]
]

# Conversion en float (les valeurs invalides deviennent NaN)
df_all[cols_float] = df_all[cols_float].apply(
    lambda col: pd.to_numeric(col, errors="coerce")
)

In [14]:
# Convertir le DataFrame en CSV et résidant en mémoire
csv_buffer = io.StringIO()
df_all.to_csv(csv_buffer, index=False, encoding="utf-8")

# Upload direct sur S3
s3_key = "public/prod/eCO2mix_RTE_Annuel-Definitif.csv"

s3.Object(bucket_name, s3_key).put(Body=csv_buffer.getvalue())

print(f"CSV envoyé sur s3://{bucket_name}/{s3_key}")


CSV envoyé sur s3://renergies99-bucket/public/prod/eCO2mix_RTE_Annuel-Definitif.csv


In [15]:
df_all.shape

(243374, 40)

In [16]:
df_all.head()

,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
0,France,Données définitives,2012-01-01,00:00:00,58315.0,58200.0,58200.0,492.0,25.0,3816.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,France,Données définitives,2012-01-01,00:30:00,58315.0,57200.0,56900.0,492.0,25.0,3816.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,France,Données définitives,2012-01-01,01:00:00,56231.0,55200.0,55100.0,492.0,25.0,3834.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,France,Données définitives,2012-01-01,01:30:00,56075.0,56000.0,55400.0,491.0,25.0,3832.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,France,Données définitives,2012-01-01,02:00:00,55532.0,56000.0,55000.0,492.0,25.0,3839.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
print(df_all.info())
print(df_all.describe())
print(df_all.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 243374 entries, 0 to 486733
Data columns (total 40 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   Périmètre                             243374 non-null  object        
 1   Nature                                243360 non-null  object        
 2   Date                                  243360 non-null  datetime64[ns]
 3   Heures                                243360 non-null  object        
 4   Consommation                          243333 non-null  float64       
 5   Prévision J-1                         243360 non-null  float64       
 6   Prévision J                           243360 non-null  float64       
 7   Fioul                                 243330 non-null  float64       
 8   Charbon                               243330 non-null  float64       
 9   Gaz                                   243330 non-null  float64  